# 16. 多线程（2）

# 共享变量
- 当多个线程同时访问同一个变量时，就会产生共享变量问题
- 解决方法：锁、信号灯等方法

In [1]:
# 共享变量问题示例
# 一般情况下
import threading

sum = 0
loopSum = 1000000

def myAdd():
    global  sum, loopSum
    for i in range(1, loopSum):
        sum += 1

def myMinu():
    global  sum, loopSum
    for i in range(1, loopSum):
        sum -= 1

if __name__ == '__main__':
    myAdd()
    print(sum)
    myMinu()
    print(sum)

999999
0


In [12]:
# 共享变量问题示例
# 多线程情况下
import threading

sum = 0
loopSum = 1000000

def myAdd():
    global  sum, loopSum
    for i in range(1, loopSum):
        sum += 1

def myMinu():
    global  sum, loopSum
    for i in range(1, loopSum):
        sum -= 1

if __name__ == '__main__':
    print("Starting ....{0}".format(sum))

    # 开始多线程的实例，看执行结果是否一样
    t1 = threading.Thread(target=myAdd, args=())
    t2 = threading.Thread(target=myMinu, args=())

    t1.start()
    t2.start()

    t1.join()
    t2.join()

    print("Done .... {0}".format(sum))
    # 由于加减运算在计算机中不是原子操作（即一步完成的操作），所以多线程会导致结果不理想

Starting ....0
Done .... 40972


## 线程锁
- 锁分为**互斥锁**和**非互斥锁**
    - 互斥锁如果嵌套了多个锁之后，会将自己锁死永远都出不来了
    - 非互斥锁：为了解决死锁，这个时候可以使用 递归锁 ，它相当于一个字典，记录了锁的门与锁的对应值，当开门的时候会根据对应来开锁。
- 锁应理解成是一个标志，表示一个线程在占用一些资源
- 原始锁是一个在锁定时不属于特定线程的同步基元组件
- 在Python中，它是能用的最低级的同步基元组件，由 _thread 扩展模块直接实现
- 原始锁处于 "锁定" 或者 "非锁定" 两种状态之一，它被创建时为非锁定状态
- Lock()：实现原始锁对象的类。一旦一个线程获得一个锁，会阻塞随后尝试获得锁的线程，直到它被释放；任何线程都可以释放它。
    - 格式：threading.Lock()
    - 需要注意的是 Lock 其实是一个工厂函数，返回平台支持的具体锁类中最有效的版本的实例
- acquire()：可以阻塞或非阻塞地获得锁
    - 格式：acquire(blocking=True, timeout=-1)
    - 参数：
        - 当调用时参数 blocking 设置为 True （缺省值），阻塞直到锁被释放，然后将锁锁定并返回 True 。
        - 在参数 blocking 被设置为 False 的情况下调用，将不会发生阻塞。
        - 如果调用时 blocking 设为 True 会阻塞，并立即返回 False ；否则，将锁锁定并返回 True。
        - 当浮点型 timeout 参数被设置为正值调用时，只要无法获得锁，将最多阻塞 timeout 设定的秒数。
        - timeout 参数被设置为 -1 时将无限等待。当 blocking 为 false 时，timeout 指定的值将被忽略。
    - 返回值：如果成功获得锁，则返回 True，否则返回 False (例如发生 超时 的时候)。
- release()：释放一个锁。这个方法可以在任何线程中调用，不单指获得锁的线程
    - 格式：release()
    - 返回值：无
    - 只有当前进程或线程是锁的持有者时，才允许调用这个方法。
    - 当锁被锁定，将它重置为未锁定，并返回。如果其他线程正在等待这个锁解锁而被阻塞，只允许其中一个线程。
    - 在未锁定的锁调用时，会引发 RuntimeError 异常。
- locked()：如果获得了锁则返回真值
- 使用方法：
    - 生成实例
        - 格式：lock = threading.Lock()
    - 上锁
        - 格式：lock.acquire()
    - 使用共享资源，随意使用
    - 解锁
        - 格式：lock.release()
    - 锁的对象：多线程中“互斥的共享资源”

In [7]:
# 不加锁的时候的数据变量安全问题
from threading import Thread

n = 0

def func1():
    global n
    for i in range(1500000):  # 循环次数少的话出错概率较低
        n += 1

def func2():
    global n
    for i in range(1500000):
        n -= 1

t1 = Thread(target=func1)
t2 = Thread(target=func2)
t1.start()
t2.start()
thread_list.append(t1)
thread_list.append(t2)

t1.join()
t2.join()

print('最后结果为：', n)

# 若不存在数据问题，则 n +1500000 -1500000 应为 n = 0

最后结果为： -467990


### 互斥锁（Lock）

In [14]:
# 用锁解决共享变量问题示例
import threading

sum = 0
loopSum = 1000000

lock = threading.Lock()

def myAdd():
    global sum, loopSum
    for i in range(1, loopSum):
        # 上锁
        lock.acquire()
        sum += 1
        # 解锁
        lock.release()

def myMinu():
    global  sum, loopSum
    for i in range(1, loopSum):
        # 上锁
        lock.acquire()
        sum -= 1
        # 解锁
        lock.release()

if __name__ == '__main__':
    print("Starting ....{0}".format(sum))

    t1 = threading.Thread(target=myAdd, args=())
    t2 = threading.Thread(target=myMinu, args=())

    t1.start()
    t2.start()

    t1.join()
    t2.join()

    print("Done .... {0}".format(sum))

Starting ....0
Done .... 0


### 死锁现象
- 例子：
    - 假如有多个人吃一碗面，吃面需要两个物品：面，筷子
    - 当其中某个人同时抢到了面和筷子时，才可以吃面，吃了面才会把面和筷子重新放下
    - 现在出现一个问题，有两个人，一个人抢到了面，一个人抢到了筷子，则两个人都无法吃面，也就都不会把面和筷子放下
    - 在这样的情况下，就无法继续进行下去，就造成了死锁现象
- 死锁现象不是互斥锁本身存在的问题，主要形成原因为程序员的编程逻辑问题
- 锁的释放顺序很重要
- 死锁发生条件：
    1. 有两个或两个以上数量的锁
    2. 多个线程异步执行
    3. 线程需要同时抢到多把锁才可以执行
    3. 多个线程同时抢到不同的锁

In [3]:
# 死锁现象的例子
from threading import Thread, Lock
import time

noodles_lock = Lock()  # 面条锁
chop_lock = Lock()  # 筷子锁

# 注意加锁和释放锁的顺序

def thread1(name):
    noodles_lock.acquire()
    print(f'{name}拿到面')
    chop_lock.acquire()
    print(f'{name}拿到筷子')
    
    print(f'{name}在吃面...')
    time.sleep(3)
    
    chop_lock.release()
    print(f'{name}放下筷子')
    noodles_lock.release()
    print(f'{name}放下面')
    print()

def thread2(name):
    chop_lock.acquire()
    print(f'{name}拿到筷子')
    noodles_lock.acquire()
    print(f'{name}拿到面')
    
    print(f'{name}在吃面...')
    time.sleep(3)
    
    noodles_lock.release()
    print(f'{name}放下面')
    chop_lock.release()
    print(f'{name}放下筷子')
    print()

namelist1 = ['三玖', '四叶']
namelist2 = ['二乃', '五月']

# 第一组线程
for name in namelist1:
    Thread(target=thread1, args=(name,)).start()
# 第二组线程
for name in namelist2:
    Thread(target=thread2, args=(name,)).start()

三玖拿到面
三玖拿到筷子
三玖在吃面...
三玖放下筷子二乃拿到筷子

三玖放下面

四叶拿到面


#### 对以上代码的说明

- 第一组线程的加锁和释放锁的顺序与第二组不同（如果顺序相同，这里就不会因为争抢而造成死锁。因为锁的获取是有顺序的，以第一组线程为例，获取不到面条锁，就无法继续执行获取筷子锁）。

- 两组线程如果分别在不同进程中运行，则都不会出问题。

- 但是在同一进程中，各线程会争抢先被释放的锁，第一组线程中的一个线程最先释放出的 筷子锁 会被第二组线程获取，后释放的 面条锁 会被第一组线程中的另外一个线程获取。

- 当一个线程必须拥有两把锁才能继续执行的时候，就会出现死锁，逻辑出问题。

### 递归锁（RLock）
- 递归锁其实只是用了同一把锁

In [2]:
# 递归锁解决上面的死锁问题
from threading import Thread, RLock
import time

noodles_rlock = chop_rlock = RLock()  # 面条锁 和 筷子锁 为同一把锁

# 注意加锁和释放锁的顺序

def thread1(name):
    noodles_rlock.acquire()
    print(f'{name}拿到面')
    chop_rlock.acquire()
    print(f'{name}拿到筷子')
    
    print(f'{name}在吃面...')
    time.sleep(3)
    
    chop_rlock.release()
    print(f'{name}放下筷子')
    noodles_rlock.release()
    print(f'{name}放下面')
    print()

def thread2(name):
    chop_rlock.acquire()
    print(f'{name}拿到筷子')
    noodles_rlock.acquire()
    print(f'{name}拿到面')
    
    print(f'{name}在吃面...')
    time.sleep(3)
    
    noodles_rlock.release()
    print(f'{name}放下面')
    chop_rlock.release()
    print(f'{name}放下筷子')
    print()

namelist1 = ['三玖', '四叶']
namelist2 = ['二乃', '五月']

# 第一组线程
for name in namelist1:
    Thread(target=thread1, args=(name,)).start()
# 第二组线程
for name in namelist2:
    Thread(target=thread2, args=(name,)).start()

三玖拿到面
三玖拿到筷子
三玖在吃面...
三玖放下筷子
三玖放下面

四叶拿到面
四叶拿到筷子
四叶在吃面...
四叶放下筷子
四叶放下面

二乃拿到筷子
二乃拿到面
二乃在吃面...
二乃放下面
二乃放下筷子

五月拿到筷子
五月拿到面
五月在吃面...
五月放下面
五月放下筷子



## 线程安全问题
- 如果一个资源或变量，它对于多线程来讲，不用加锁也不会引起任何问题，则称为线程安全
- 线程不安全变量类型：list, set, dict
- 线程安全变量类型：queue（一个同步的队列类）

## 生产者消费者问题
- 一个模型，可以用来搭建消息队列， 
- queue：是一个用来存放变量的数据结构，特点是先进先出，内部元素排队，可以理解成一个特殊的list
- queue 模块实现了多生产者、多消费者队列。这特别适用于消息必须安全地在多线程间交换的线程编程。模块中的 Queue 类实现了所有所需的锁定语义。
- qsize()：返回队列的大致大小。注意，qsize() > 0 不保证后续的 get() 不被阻塞，qsize() < maxsize 也不保证 put() 不被阻塞。
- empty()：如果队列为空，返回 True ，否则返回 False 。如果 empty() 返回 True ，不保证后续调用的 put() 不被阻塞。类似的，如果 empty() 返回 False ，也不保证后续调用的 get() 不被阻塞。
- full()：如果队列是满的返回 True ，否则返回 False 。如果 full() 返回 True 不保证后续调用的 get() 不被阻塞。类似的，如果 full() 返回 False 也不保证后续调用的 put() 不被阻塞。
- put()：将 item 放入队列。
    - 格式：put(item, block=True, timeout=None)
    - 如果可选参数 block 是 true 并且 timeout 是 None (默认)，则在必要时阻塞至有空闲插槽可用。
    - 如果 timeout 是个正数，将最多阻塞 timeout 秒，如果在这段时间没有可用的空闲插槽，将引发 Full 异常。
    - 反之 (block 是 false)，如果空闲插槽立即可用，则把 item 放入队列，否则引发 Full 异常 ( 在这种情况下，timeout 将被忽略)。
- get()：从队列中移除并返回一个项目。
    - 格式：get(block=True, timeout=None)
    - 如果可选参数 block 是 true 并且 timeout 是 None (默认值)，则在必要时阻塞至项目可得到。如果 timeout 是个正数，将最多阻塞 timeout 秒，如果在这段时间内项目不能得到，将引发 Empty 异常。
    - 如果可选参数 block 是 false, 如果一个项目立即可得到，则返回一个项目，否则引发 Empty 异常 (这种情况下，timeout 将被忽略)。

In [15]:
# 生产消费者模型示例
#encoding=utf-8
import threading
import time

# 导入queue模块
import queue

# 生产者
class Producer(threading.Thread):
    def run(self):
        global que
        count = 0
        while True:
            # qsize() 返回que内容长度
            if que.qsize() < 1000:
                for i in range(100):
                    count = count + 1
                    msg = '生成产品' + str(count)
                    # put() 是向que中放入一个值
                    que.put(msg)
                    print(msg)
            time.sellp(0.5)

# 消费者
class Consumer(threading.Thread):
    def run(self):
        global que
        while True:
            # qsize() 返回queue内容长度
            if que.qsize() > 100:
                for i in range(3):
                    # get() 是从que中取出一个值
                    msg = self.name + '消费了 ' + que.get()
                    print(msg)
            time.sleep(1)

if __name__ == '__main__':
    que = queue.Queue()

    for i in range(500):
        que.put('初始产品'+str(i))
    for i in range(2):
        p = Producer()
        p.start()
    for i in range(5):
        c = Consumer()
        c.start()

# 因为是无限运行，所以强制终止会报错

生成产品1生成产品1
生成产品2
生成产品3
生成产品4
生成产品5

生成产品2
生成产品3
生成产品4
生成产品5
生成产品6
生成产品7
生成产品6
生成产品7
生成产品8
生成产品9
Thread-34消费了 初始产品0
Thread-34消费了 初始产品1
Thread-34消费了 初始产品2
生成产品8
生成产品9
生成产品10
生成产品11
生成产品12
生成产品13
生成产品14
生成产品15
生成产品16
生成产品17
生成产品18
生成产品19
生成产品20
生成产品21
生成产品10生成产品22
生成产品23

生成产品11
生成产品12
生成产品13
生成产品24Thread-35消费了 初始产品3
Thread-35消费了 初始产品4
生成产品14Thread-35消费了 初始产品5


生成产品15
生成产品16
生成产品25
生成产品26
生成产品27
生成产品28
生成产品29
生成产品30
生成产品31
生成产品17生成产品32
生成产品33
生成产品34
生成产品35

Thread-36消费了 初始产品6
生成产品36
生成产品37Thread-36消费了 初始产品7
Thread-36消费了 初始产品8
Thread-37消费了 初始产品9
生成产品38
生成产品39
生成产品40
生成产品41
生成产品42
生成产品43
生成产品44
生成产品18
Thread-37消费了 初始产品10
生成产品45
生成产品46
生成产品47

生成产品19
生成产品20
生成产品21
生成产品22
生成产品23
生成产品24
生成产品25
生成产品26
生成产品27
生成产品28
Thread-37消费了 初始产品11
生成产品48Thread-38消费了 初始产品12
生成产品29
Thread-38消费了 初始产品13
Thread-38消费了 初始产品14
生成产品49
生成产品50
生成产品30
生成产品31
生成产品32
生成产品33
生成产品34

生成产品51
生成产品52
生成产品53
生成产品54
生成产品55
生成产品56
生成产品57
生成产品58
生成产品59
生成产品60
生成产品61
生成产品62
生成产品63
生成产品64
生成产品65
生成产品66
生成产品67
生成产品68
生成产品35
生成产品69


Exception in thread Thread-33:
Traceback (most recent call last):
  File "G:\Anaconda3\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "<ipython-input-15-f91fb01797a8>", line 23, in run
    time.sellp(0.5)
AttributeError: module 'time' has no attribute 'sellp'

Exception in thread Thread-32:
Traceback (most recent call last):
  File "G:\Anaconda3\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "<ipython-input-15-f91fb01797a8>", line 23, in run
    time.sellp(0.5)
AttributeError: module 'time' has no attribute 'sellp'



Thread-34消费了 初始产品15
Thread-34消费了 初始产品16
Thread-34消费了 初始产品17
Thread-35消费了 初始产品18
Thread-35消费了 初始产品19
Thread-35消费了 初始产品20
Thread-36消费了 初始产品21Thread-37消费了 初始产品22
Thread-37消费了 初始产品23
Thread-37消费了 初始产品24

Thread-36消费了 初始产品25
Thread-36消费了 初始产品26
Thread-38消费了 初始产品27
Thread-38消费了 初始产品28
Thread-38消费了 初始产品29
Thread-34消费了 初始产品30
Thread-34消费了 初始产品31
Thread-34消费了 初始产品32
Thread-35消费了 初始产品33
Thread-35消费了 初始产品34
Thread-35消费了 初始产品35
Thread-36消费了 初始产品36Thread-37消费了 初始产品37
Thread-37消费了 初始产品38
Thread-37消费了 初始产品39

Thread-36消费了 初始产品40
Thread-36消费了 初始产品41
Thread-38消费了 初始产品42
Thread-38消费了 初始产品43
Thread-38消费了 初始产品44
Thread-34消费了 初始产品45
Thread-34消费了 初始产品46
Thread-34消费了 初始产品47
Thread-35消费了 初始产品48
Thread-35消费了 初始产品49
Thread-35消费了 初始产品50
Thread-37消费了 初始产品51Thread-36消费了 初始产品52
Thread-36消费了 初始产品53
Thread-36消费了 初始产品54

Thread-37消费了 初始产品55
Thread-37消费了 初始产品56
Thread-38消费了 初始产品57
Thread-38消费了 初始产品58
Thread-38消费了 初始产品59
Thread-34消费了 初始产品60
Thread-34消费了 初始产品61
Thread-34消费了 初始产品62
Thread-35消费了 初始产品63
Thread-35消费了 初始产品64


## 死锁问题
- 多线程中一定要避免死锁问题。
- 解锁顺序一定要与上锁顺序相反

In [ ]:
# 死锁问题示例
import time
import threading

lock_1 = threading.Lock()
lock_2 = threading.Lock()


def func_1():
    print("func_1 starting.........")
    
    lock_1.acquire()
    print("func_1 申请了 lock_1....")
    
    time.sleep(2)
    print("func_1 等待 lock_2.......")
    
    lock_2.acquire()
    print("func_1 申请了 lock_2.......")
    
    # 解锁顺序一定要与上锁顺序相反，所以此处要先释放2号锁
    lock_2.release()
    print("func_1 释放了 lock_2")
    
    lock_1.release()
    print("func_1 释放了 lock_1")

    print("func_1 done..........")

def func_2():
    print("func_2 starting.........")
    
    lock_2.acquire()
    print("func_2 申请了 lock_2....")
    
    time.sleep(4)
    print("func_2 等待 lock_1.......")
    
    lock_1.acquire()
    print("func_2 申请了 lock_1.......")
    
    # 解锁顺序一定要与上锁顺序相反，所以此处要先释放1号锁
    lock_1.release()
    print("func_2 释放了 lock_1")

    lock_2.release()
    print("func_2 释放了 lock_2")

    print("func_2 done..........")


if __name__ == "__main__":
    print("主程序启动..............")
    
    t1 = threading.Thread(target=func_1, args=())
    t2 = threading.Thread(target=func_2, args=())
    
    t1.start()
    t2.start()
    
    t1.join()
    t2.join()
    
    print("主程序结束..............")

# 此处死锁

主程序启动..............
func_1 starting.........
func_1 申请了 lock_1....
func_2 starting.........
func_2 申请了 lock_2....
func_1 等待 lock_2.......
func_2 等待 lock_1.......


## 死锁的解决方法

### 设置上锁的等待时间
- 设置申请等待时间，如果申请不到，就释放自己

In [2]:
# 死锁解决示例1
import time
import threading

lock_1 = threading.Lock()
lock_2 = threading.Lock()

def func_1():# 此段代码有瑕疵：假如lock_1没有申请到，就会报错
    print("func_1 starting.........")
    
    lock_1.acquire(timeout=4)
    print("func_1 申请了 lock_1....")
    time.sleep(2)
    print("func_1 等待 lock_2.......")
    
    rst = lock_2.acquire(timeout=2)
    if rst:
        print("func_1 已经得到锁 lock_2")
        lock_2.release()
        print("func_1 释放了锁 lock_2")
    else:
        print("func_1 注定没申请到lock_2.....")
    
    lock_1.release()
    print("func_1 释放了 lock_1")
    
    print("func_1 done..........")

def func_2():# 此段代码有瑕疵：假如lock_1没有申请到，就会报错
    print("func_2 starting.........")
    lock_2.acquire()
    print("func_2 申请了 lock_2....")
    time.sleep(4)
    print("func_2 等待 lock_1.......")
    lock_1.acquire()
    print("func_2 申请了 lock_1.......")

    lock_1.release()
    print("func_2 释放了 lock_1")

    lock_2.release()
    print("func_2 释放了 lock_2")

    print("func_2 done..........")


if __name__ == "__main__":
    print("主程序启动..............")
    
    t1 = threading.Thread(target=func_1, args=())
    t2 = threading.Thread(target=func_2, args=())
    
    t1.start()
    t2.start()
    
    t1.join()
    t2.join()
    
    print("主程序结束..............")

# 此程序不会有死锁问题

主程序启动..............
func_1 starting.........
func_1 申请了 lock_1....
func_2 starting.........
func_2 申请了 lock_2....
func_1 等待 lock_2.......
func_1 注定没申请到lock_2.....
func_1 释放了 lock_1
func_1 done..........
func_2 等待 lock_1.......
func_2 申请了 lock_1.......
func_2 释放了 lock_1
func_2 释放了 lock_2
func_2 done..........
主程序结束..............


### Semaphore（线程信号量） - 多线程争抢多资源解决方案
- Semaphore：该类实现信号量对象。允许一个资源最多被多少个线程同时使用。
    - 可选参数 value 赋予内部计数器初始值，默认值为 1 。
    - 如果 value 被赋予小于0的值，将会引发 ValueError 异常。
- 信号量对象管理一个原子性的计数器，代表 release() 方法的调用次数减去 acquire() 的调用次数再加上一个初始值。如果需要， acquire() 方法将会阻塞直到可以返回而不会使得计数器变成负数。在没有显式给出 value 的值时，默认为1。
- acquire()：获取一个信号量。
    - 格式：acquire(blocking=True, timeout=None)
    - 在不带参数的情况下调用时：
        - 如果在进入时内部计数器的值大于零，则将其减一并立即返回 True。
        - 如果在进入时内部计数器的值为零，则将会阻塞直到被对 release() 的调用唤醒。 一旦被唤醒（并且计数器的值大于 0），则将计数器减 1 并返回 True。 每次对 release() 的调用将只唤醒一个线程。 线程被唤醒的次序是不可确定的。
    - 在带参数的情况下调用时：
        - 当发起调用时将 blocking 设为假值，则不进行阻塞。 如果一个无参数调用将要阻塞，则立即返回 False；在其他情况下，执行与无参数调用时一样的操作，然后返回 True。
        - 当发起调用时如果 timeout 不为 None，则它将阻塞最多 timeout 秒。 请求在此时段时未能成功完成获取则将返回 False。 在其他情况下返回 True。
- release()：释放一个信号量，将内部计数器的值增加1。当计数器原先的值为0且有其它线程正在等待它再次大于0时，唤醒正在等待的线程

In [3]:
# 死锁解决示例2
import time
import threading

# 参数定义最多几个线程同时使用资源
semaphore = threading.Semaphore(3)

def func():
    if semaphore.acquire():
        for i in range(5):
            print(threading.currentThread().getName() + ' get semaphore')
        time.sleep(15)
        semaphore.release()
        print(threading.currentThread().getName() + ' release semaphore')


for i in range(8):
    t1 = threading.Thread(target=func)
    t1.start()

Thread-10 get semaphore
Thread-10 get semaphore
Thread-10 get semaphore
Thread-10 get semaphore
Thread-10 get semaphore
Thread-11 get semaphore
Thread-11 get semaphore
Thread-11 get semaphoreThread-12 get semaphore
Thread-12 get semaphore
Thread-12 get semaphore
Thread-12 get semaphore
Thread-12 get semaphore

Thread-11 get semaphore
Thread-11 get semaphore
Thread-10 release semaphoreThread-13 get semaphore
Thread-13 get semaphore
Thread-13 get semaphore
Thread-13 get semaphore
Thread-13 get semaphore

Thread-12 release semaphoreThread-14 get semaphore
Thread-14 get semaphore
Thread-14 get semaphore
Thread-11 release semaphore

Thread-14 get semaphore
Thread-14 get semaphore
Thread-15 get semaphore
Thread-15 get semaphore
Thread-15 get semaphore
Thread-15 get semaphore
Thread-15 get semaphore
Thread-13 release semaphoreThread-16 get semaphore
Thread-16 get semaphore

Thread-16 get semaphore
Thread-16 get semaphore
Thread-16 get semaphore
Thread-14 release semaphoreThread-17 get semapho

### threading.Timer - 定时器对象
- 此类表示一个操作应该在等待一定的时间之后运行 --- 相当于一个定时器。 Timer 类是 Thread 类的子类，因此可以像一个自定义线程一样工作。
- Timer是利用多线程，在指定时间后启动一个功能
- 与线程一样，通过调用 start() 方法启动定时器。而 cancel() 方法可以停止计时器（在计时结束前）， 定时器在执行其操作之前等待的时间间隔可能与用户指定的时间间隔不完全相同。
- Timer()：
    - 格式：threading.Timer(interval, function, args=None, kwargs=None)
    - 创建一个定时器，在经过 interval 秒的间隔事件后，将会用参数 args 和关键字参数 kwargs 调用 function。
    - 如果 args 为 None （默认值），则会使用一个空列表。
    - 如果 kwargs 为 None （默认值），则会使用一个空字典。
- cancel()：停止定时器并取消执行计时器将要执行的操作。仅当计时器仍处于等待状态时有效。

In [4]:
# Timer示例
import time
import threading

def func():
    print("I am running.........")
    time.sleep(4)
    print("I am done......")

if __name__ == "__main__":
    t = threading.Timer(6, func)
    t.start()

    i = 0
    while True:
        print("{0}***************".format(i))
        time.sleep(3)
        i += 1

0***************
1***************
I am running.........2***************

3***************
I am done......
4***************
5***************
6***************
7***************
8***************
9***************
10***************
11***************
12***************
13***************
14***************
15***************
16***************


KeyboardInterrupt: 

### 可重入锁
- 重入锁是一个可以被同一个线程多次获取的同步基元组件。
- 主要解决递归调用的时候，需要申请锁的情况。
- 在内部，它在基元锁的锁定/非锁定状态上附加了 "所属线程" 和 "递归等级" 的概念。
- 在锁定状态下，某些线程拥有锁 ； 在非锁定状态下， 没有线程拥有它。
- 若要锁定锁，线程调用其 acquire() 方法；一旦线程拥有了锁，方法将返回。若要解锁，线程调用 release() 方法。
- acquire()/release() 对可以嵌套；只有最终 release() (最外面一对的 release() ) 将锁解开，才能让其他线程继续处理 acquire() 阻塞。
- acquire(blocking=True, timeout=-1)：可以阻塞或非阻塞地获得锁。
    - 当无参数调用时： 如果这个线程已经拥有锁，递归级别增加一，并立即返回。否则，如果其他线程拥有该锁，则阻塞至该锁解锁。一旦锁被解锁(不属于任何线程)，则抢夺所有权，设置递归等级为一，并返回。如果多个线程被阻塞，等待锁被解锁，一次只有一个线程能抢到锁的所有权。在这种情况下，没有返回值。
    - 当发起调用时将 blocking 参数设为真值，则执行与无参数调用时一样的操作，然后返回 True。
    - 当发起调用时将 blocking 参数设为假值，则不进行阻塞。 如果一个无参数调用将要阻塞，则立即返回 False；在其他情况下，执行与无参数调用时一样的操作，然后返回 True。
    - 当发起调用时将浮点数的 timeout 参数设为正值时，只要无法获得锁，将最多阻塞 timeout 所指定的秒数。 如果已经获得锁则返回 True，如果超时则返回假值。
- release()：释放锁，自减递归等级。
    - 如果减到零，则将锁重置为非锁定状态(不被任何线程拥有)，并且，如果其他线程正被阻塞着等待锁被解锁，则仅允许其中一个线程继续。如果自减后，递归等级仍然不是零，则锁保持锁定，仍由调用线程拥有。
    - 只有当前线程拥有锁才能调用这个方法。如果锁被释放后调用这个方法，会引起 RuntimeError 异常。
    - 没有返回值。

In [1]:
# 重入锁 示例
import time
import threading

class MyThread(threading.Thread):
    def run(self):
        global num
        time.sleep(1)

        if mutex.acquire(1):
            num = num + 1
            msg = self.name + ' set num to ' + str(num)
            print(msg)
            mutex.acquire()
            mutex.release()
            mutex.release()

num = 0

# 当此处为“mutex = threading.Lock()”时为死锁
mutex = threading.RLock()

def testTh():
    for i in range(5):
        t = MyThread()
        t.start()



if __name__ == '__main__':
    testTh()

Thread-6 set num to 1
Thread-9 set num to 2
Thread-8 set num to 3
Thread-10 set num to 4
Thread-7 set num to 5


## 线程事件

In [3]:
# 有两个线程异步执行
# 假设一个线程用于检测与服务器之间的连接
# 另外一个线程用于显示连接的结果
from threading import Thread, Event
import time

# 连接状态检测函数
def check(e):
    print('正在检测与服务器的连接...')
    time.sleep(4)  # 假设此处进行的是真正的检测
    e.set()  # 检测成功后将 Eevent 对象的内部标识设置为 True


# 连接结果显示函数
def connect(e):
    # 三次机会
    for i in range(3):
        # 假设每次机会只有1.5秒时间
        e.wait(1.5)
        # 检查 Eevent 对象的内部标识
        if e.is_set():  # Eevent 对象的内部标识为 True 时
            print('与服务器连接成功！')
            break
        else:  # Eevent 对象的内部标识为 False 时
            print(f'第{i+1}次与服务器连接失败！')


# 创建事件对象
e = Event()
# 创建两个异步线程
Thread(target=connect, args=(e,)).start()
Thread(target=check, args=(e,)).start()

正在检测与服务器的连接...
第1次与服务器连接失败！
第2次与服务器连接失败！
与服务器连接成功！


## 线程条件
- wait() 和 notify() 方法需要配合使用
- wait() 和 notify() 方法前后要加锁和释放锁

In [9]:
# 控制每次执行线程的数量
from threading import Thread, Condition

def threads(cond, i):
    cond.acquire()
    
    cond.wait()
    print(f'{i}号线程执行。')
    
    cond.release()


cond = Condition()

count = 10

# 设置10个线程
for i in range(count):
    t = Thread(target=threads, args=(cond, i))
    t.start()

# 使用输入的数字来控制每次执行线程的个数
while count > 0:
    num = int(input('请输入要执行线程的个数：'))
    
    cond.acquire()
    
    cond.notify(num)
    count -= num
    
    cond.release()

请输入要执行线程的个数： 2


0号线程执行。
1号线程执行。


请输入要执行线程的个数： 5


2号线程执行。
3号线程执行。
4号线程执行。
5号线程执行。
6号线程执行。


请输入要执行线程的个数： 3


9号线程执行。
7号线程执行。
8号线程执行。


## 线程池